# Distribution of influences

To get a grip on how much randomness influences the results, we run the same model lots of times and compare the results.

In [1]:
# for Colab, install fin_benefits and unemployment-gym from Github
#!pip install -q git+https://github.com/ajtanskanen/benefits.git  
#!pip install -q git+https://github.com/ajtanskanen/econogym.git
#!pip install -q git+https://github.com/ajtanskanen/lifecycle-rl.git

# and then restart kernel
  
  # For a specific version:
#!pip install tensorflow==1.15
#!pip install stable-baselines==2.8
  
# restart kernel after running pip's

Then load all modules and set parameters for simulations.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from lifecycle_rl import Lifecycle

%matplotlib inline
%pylab inline

# varoitukset piiloon (Stable baseline ei ole vielä Tensorflow 2.0-yhteensopiva, ja Tensorflow 1.15 valittaa paljon)
# ei taida toimia piilottaminen
import warnings
warnings.filterwarnings('ignore')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Populating the interactive namespace from numpy and matplotlib


In [2]:
# parameters for the simulation
# episode = 51 / 205 timesteps (1y/3m timestep)
pop_size=20_000 # size of the population to be simulated
size1=6_000_000 #0_000 # number of timesteps in phase 1 training (callback not used)
size2=100 #0_000 # number of timesteps in phase 2 training (callback is used to save the best results)
size3=100 # number of timesteps in phase 1 training (callback not used) for policy changes
batch1=1_00 # size of minibatch in phase 1 as number of episodes
batch2=9_00  # size of minibatch in phase 1 as number of episodes
callback_minsteps=batch2 # how many episodes callback needs 
deterministic=False # use deterministic prediction (True) or probabilitic prediction (False)
mortality=False # include mortality in computations
randomness=True # include externally given, random state-transitions (parental leaves, disability, lay-offs) 
pinkslip=True # include lay-offs at 5 percent level each year
rlmodel='acktr' # use ACKTR algorithm
twostage=False # ajataan kahdessa vaiheessa vai ei
perusmalli='best/malli_baseline'


# Baseline

Lasketaan työllisyysasteet nykymallissa.

In [4]:
cc1=Lifecycle(env='unemployment-v1',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=False)
cc1.explain()
cc1.run_distrib(n=50,startn=17,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,
                save=perusmalli,plot=True,cont=True,start_from=perusmalli,results='results/distrib_base',
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage)

No mortality included
Parameters of lifecycle:
timestep 0.25
gamma 0.9793703613355593 (0.9200000000000003 per anno)
min_age 20
max_age 70
min_retirementage 63.5
max_retirementage 68
ansiopvraha_kesto300 300
ansiopvraha_kesto400 400
ansiopvraha_toe 0.5
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki True
include_pinkslip True

Parameters of lifecycle:
timestep 0.25
gamma 0.92 per anno
min_age 20
max_age 70
min_retirementage 63.5
max_retirementage 68
ansiopvraha_kesto300 None
ansiopvraha_kesto400 None
ansiopvraha_toe None
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki None
include_pinkslip True

computing 0
train...
phase 1
batch 9 learning rate 0.0625 scaled 0.1875




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






training...




---------------------------------
| explained_variance | 0.987    |
| fps                | 3477     |
| nupdates           | 50       |
| policy_entropy     | 0.997    |
| policy_loss        | -0.0783  |
| total_timesteps    | 1069033  |
| value_loss         | 0.172    |
---------------------------------
---------------------------------
| explained_variance | 0.991    |
| fps                | 3473     |
| nupdates           | 60       |
| policy_entropy     | 1.01     |
| policy_loss        | -0.0827  |
| total_timesteps    | 1287203  |
| value_loss         | 0.112    |
---------------------------------
---------------------------------
| explained_variance | 0.993    |
| fps                | 3471     |
| nupdates           | 70       |
| policy_entropy     | 0.993    |
| policy_loss        | -0.0207  |
| total_timesteps    | 1505373  |
| value_loss         | 0.0818   |
---------------------------------
---------------------------------
| explained_variance | 0.996    |
| fps         

computing 1
train...
phase 1
batch 9 learning rate 0.0625 scaled 0.1875
training...
---------------------------------
| explained_variance | 0.999    |
| fps                | 3284     |
| nupdates           | 1        |
| policy_entropy     | 1        |
| policy_loss        | -0.0174  |
| total_timesteps    | 0        |
| value_loss         | 0.0148   |
---------------------------------
---------------------------------
| explained_variance | 0.827    |
| fps                | 3614     |
| nupdates           | 10       |
| policy_entropy     | 0.997    |
| policy_loss        | 0.851    |
| total_timesteps    | 196353   |
| value_loss         | 2.88     |
---------------------------------
---------------------------------
| explained_variance | 0.962    |
| fps                | 3555     |
| nupdates           | 20       |
| policy_entropy     | 0.991    |
| policy_loss        | -0.0871  |
| total_timesteps    | 414523   |
| value_loss         | 0.465    |
--------------------------------

---------------------------------
| explained_variance | 0.999    |
| fps                | 3502     |
| nupdates           | 270      |
| policy_entropy     | 1.01     |
| policy_loss        | -0.0104  |
| total_timesteps    | 5868773  |
| value_loss         | 0.0107   |
---------------------------------
done
simulating  best/malli_baseline_v101
predicting...


computing 2
train...
phase 1
batch 9 learning rate 0.0625 scaled 0.1875
training...
---------------------------------
| explained_variance | 0.999    |
| fps                | 3306     |
| nupdates           | 1        |
| policy_entropy     | 1.03     |
| policy_loss        | -0.0175  |
| total_timesteps    | 0        |
| value_loss         | 0.0139   |
---------------------------------
---------------------------------
| explained_variance | 0.707    |
| fps                | 3623     |
| nupdates           | 10       |
| policy_entropy     | 0.993    |
| policy_loss        | 0.706    |
| total_timesteps    | 196353   |
| value_loss         | 4.14     |
---------------------------------
---------------------------------
| explained_variance | 0.906    |
| fps                | 3579     |
| nupdates           | 20       |
| policy_entropy     | 1.02     |
| policy_loss        | -0.143   |
| total_timesteps    | 414523   |
| value_loss         | 1.12     |
--------------------------------

...

---------------------------------
| explained_variance | 0.999    |
| fps                | 3513     |
| nupdates           | 270      |
| policy_entropy     | 1.01     |
| policy_loss        | -0.01    |
| total_timesteps    | 5868773  |
| value_loss         | 0.0106   |
---------------------------------
done
simulating  best/malli_baseline_v115
predicting...


computing 16
train...
phase 1
batch 9 learning rate 0.0625 scaled 0.1875
training...
---------------------------------
| explained_variance | 0.999    |
| fps                | 3296     |
| nupdates           | 1        |
| policy_entropy     | 1.01     |
| policy_loss        | -0.00936 |
| total_timesteps    | 0        |
| value_loss         | 0.0134   |
---------------------------------
---------------------------------
| explained_variance | 0.918    |
| fps                | 3610     |
| nupdates           | 10       |
| policy_entropy     | 1.01     |
| policy_loss        | 0.653    |
| total_timesteps    | 196353   |
| value_loss         | 1.37     |
---------------------------------
---------------------------------
| explained_variance | 0.784    |
| fps                | 3582     |
| nupdates           | 20       |
| policy_entropy     | 0.993    |
| policy_loss        | -1.03    |
| total_timesteps    | 414523   |
| value_loss         | 3.75     |
-------------------------------

---------------------------------
| explained_variance | 0.999    |
| fps                | 3500     |
| nupdates           | 270      |
| policy_entropy     | 1.02     |
| policy_loss        | -0.01    |
| total_timesteps    | 5868773  |
| value_loss         | 0.0125   |
---------------------------------
done
simulating  best/malli_baseline_v116
predicting...


computing 17
train...
phase 1


KeyboardInterrupt: 

In [ ]:
cc2=Lifecycle(env='unemployment-v1',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=False)
cc2.render_distrib(load='results/distrib_base_v',n=100,stats_results='results/distrib_stats')


# Työttömyysputken poisto

Työttömyysputkelle meneminen on usein hyvin suosittua elinkaarimalleissa. Tarkastellaan millainen työllisyysvaikutus on putken poistamisella.

In [ ]:
cc1_putki=Lifecycle(env='unemployment-v1',minimal=False,include_putki=False,mortality=mortality,
                    perustulo=False,randomness=randomness)
poistomalli='best/putki/malli_putki'
cc1_putki.run_distrib(n=50,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,
                save=poistomalli,plot=True,cont=True,start_from=perusmalli,results='results/putki/distrib_poisto',
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage)
cc2.render_distrib(load='results/putki/distrib_poisto_v',n=2,startn=0,stats_results='results/putki/distrib_poisto_stats')

In [ ]:
cc1_putki_b=Lifecycle(env='unemployment-v1',minimal=False,include_putki=False,mortality=mortality,
                    perustulo=False,randomness=randomness)
#cc1_putki_b.render_distrib(load='results/putki/distrib_poisto_v')
cc1_putki_b.compare_distrib('results/putki/distrib_base_v','results/putki/distrib_poisto_v')                    


# Porrastus

In [ ]:
## cc2_porras=Lifecycle(env='unemploymentSteps-v1',minimal=False,mortality=mortality,perustulo=False,randomness=randomness)
porrasmalli='best/porrastus/malli_porrastus'
cc2_porras.run_distrib(n=50,startn=2,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=10,batch2=batch2,
                save=porrasmalli,plot=True,cont=True,start_from=perusmalli,results='results/porrastus/distrib_porras',
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage)

In [ ]:
cc2_porras_b=Lifecycle(env='unemploymentSteps-v1',minimal=False,include_putki=False,mortality=mortality,perustulo=False,randomness=randomness)
cc2_porras_b.render_distrib(load='results/porrastus/distrib_porras_v',n=1,startn=0,stats_results='results/porrastus/distrib_porras_stats')
cc2_porras_b.compare_distrib('results/distrib_stats','results/porrastus/distrib_porras_stats')



# TOE 12 kk

In [ ]:
cc2_toe=Lifecycle(env='unemployment-v1',minimal=False,ansiopvraha_toe=1.0,mortality=mortality,perustulo=False,randomness=randomness)
toemalli='best/toe12/malli_toe'
cc2_toe.run_distrib(n=50,startn=0,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=10,batch2=batch2,
                save=toemalli,plot=True,cont=True,start_from=perusmalli,results='results/toe12/distrib_toe12',
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage)

No mortality included
Parameters of lifecycle:
timestep 0.25
gamma 0.9793703613355593 (0.9200000000000003 per anno)
min_age 20
max_age 70
min_retirementage 63.5
max_retirementage 68
ansiopvraha_kesto300 300
ansiopvraha_kesto400 400
ansiopvraha_toe 1.0
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki True
include_pinkslip True

computing 0
train...
phase 1
batch 1 learning rate 0.0625 scaled 0.0625




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






training...








---------------------------------
| explained_variance | 0.999    |
| fps                | 1744     |
| nupdates           | 1        |
| policy_entropy     | 1.04     |
| policy_loss        | -0.0245  |
| total_timesteps    | 0        |
| value_loss         | 0.0126   |
------------------------------

---------------------------------
| explained_variance | 0.992    |
| fps                | 2633     |
| nupdates           | 60       |
| policy_entropy     | 0.979    |
| policy_loss        | -0.0255  |
| total_timesteps    | 143075   |
| value_loss         | 0.092    |
---------------------------------
---------------------------------
| explained_variance | 0.998    |
| fps                | 2633     |
| nupdates           | 70       |
| policy_entropy     | 0.981    |
| policy_loss        | -0.00441 |
| total_timesteps    | 167325   |
| value_loss         | 0.0307   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2623     |
| nupdates           | 80       |
| policy_entropy     | 1.03     |
| policy_loss        | -0.0151  |
| total_timesteps    | 191575   |
| value_loss         | 0.0168   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps         

---------------------------------
| explained_variance | 0.989    |
| fps                | 2610     |
| nupdates           | 330      |
| policy_entropy     | 0.994    |
| policy_loss        | -0.0414  |
| total_timesteps    | 797825   |
| value_loss         | 0.122    |
---------------------------------
---------------------------------
| explained_variance | 0.989    |
| fps                | 2605     |
| nupdates           | 340      |
| policy_entropy     | 1.01     |
| policy_loss        | 0.00851  |
| total_timesteps    | 822075   |
| value_loss         | 0.127    |
---------------------------------
---------------------------------
| explained_variance | 0.997    |
| fps                | 2607     |
| nupdates           | 350      |
| policy_entropy     | 0.965    |
| policy_loss        | -0.0147  |
| total_timesteps    | 846325   |
| value_loss         | 0.037    |
---------------------------------
---------------------------------
| explained_variance | 0.998    |
| fps         

---------------------------------
| explained_variance | 0.996    |
| fps                | 2632     |
| nupdates           | 600      |
| policy_entropy     | 1.07     |
| policy_loss        | -0.015   |
| total_timesteps    | 1452575  |
| value_loss         | 0.0434   |
---------------------------------
---------------------------------
| explained_variance | 0.996    |
| fps                | 2632     |
| nupdates           | 610      |
| policy_entropy     | 1.04     |
| policy_loss        | -0.0316  |
| total_timesteps    | 1476825  |
| value_loss         | 0.049    |
---------------------------------
---------------------------------
| explained_variance | 0.997    |
| fps                | 2633     |
| nupdates           | 620      |
| policy_entropy     | 0.988    |
| policy_loss        | 0.00181  |
| total_timesteps    | 1501075  |
| value_loss         | 0.0291   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps         

---------------------------------
| explained_variance | 0.999    |
| fps                | 2644     |
| nupdates           | 870      |
| policy_entropy     | 1.03     |
| policy_loss        | -0.00531 |
| total_timesteps    | 2107325  |
| value_loss         | 0.00788  |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2644     |
| nupdates           | 880      |
| policy_entropy     | 1        |
| policy_loss        | -0.00594 |
| total_timesteps    | 2131575  |
| value_loss         | 0.00773  |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2645     |
| nupdates           | 890      |
| policy_entropy     | 1.03     |
| policy_loss        | -0.0278  |
| total_timesteps    | 2155825  |
| value_loss         | 0.0196   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps         

...

---------------------------------
| explained_variance | 0.999    |
| fps                | 2697     |
| nupdates           | 810      |
| policy_entropy     | 1        |
| policy_loss        | -0.0108  |
| total_timesteps    | 1961825  |
| value_loss         | 0.0107   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2697     |
| nupdates           | 820      |
| policy_entropy     | 1.02     |
| policy_loss        | -0.00758 |
| total_timesteps    | 1986075  |
| value_loss         | 0.0133   |
---------------------------------
---------------------------------
| explained_variance | 0.998    |
| fps                | 2698     |
| nupdates           | 830      |
| policy_entropy     | 1.03     |
| policy_loss        | -0.0226  |
| total_timesteps    | 2010325  |
| value_loss         | 0.0252   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps         

---------------------------------
| explained_variance | 0.999    |
| fps                | 2703     |
| nupdates           | 1080     |
| policy_entropy     | 0.991    |
| policy_loss        | -0.00659 |
| total_timesteps    | 2616575  |
| value_loss         | 0.0136   |
---------------------------------
---------------------------------
| explained_variance | 0.998    |
| fps                | 2704     |
| nupdates           | 1090     |
| policy_entropy     | 1.02     |
| policy_loss        | -0.0295  |
| total_timesteps    | 2640825  |
| value_loss         | 0.0195   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2704     |
| nupdates           | 1100     |
| policy_entropy     | 0.975    |
| policy_loss        | -0.00275 |
| total_timesteps    | 2665075  |
| value_loss         | 0.0111   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps         

---------------------------------
| explained_variance | 0.999    |
| fps                | 2708     |
| nupdates           | 1350     |
| policy_entropy     | 1.03     |
| policy_loss        | -0.0237  |
| total_timesteps    | 3271325  |
| value_loss         | 0.0116   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2708     |
| nupdates           | 1360     |
| policy_entropy     | 1.04     |
| policy_loss        | -0.00954 |
| total_timesteps    | 3295575  |
| value_loss         | 0.0127   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2708     |
| nupdates           | 1370     |
| policy_entropy     | 1.06     |
| policy_loss        | -0.011   |
| total_timesteps    | 3319825  |
| value_loss         | 0.00952  |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps         

---------------------------------
| explained_variance | 0.999    |
| fps                | 2713     |
| nupdates           | 1620     |
| policy_entropy     | 0.961    |
| policy_loss        | -0.00455 |
| total_timesteps    | 3926075  |
| value_loss         | 0.00747  |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2713     |
| nupdates           | 1630     |
| policy_entropy     | 1.11     |
| policy_loss        | -0.017   |
| total_timesteps    | 3950325  |
| value_loss         | 0.00738  |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2713     |
| nupdates           | 1640     |
| policy_entropy     | 1.04     |
| policy_loss        | -0.0192  |
| total_timesteps    | 3974575  |
| value_loss         | 0.00769  |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps         

---------------------------------
| explained_variance | 0.999    |
| fps                | 2712     |
| nupdates           | 1890     |
| policy_entropy     | 1.05     |
| policy_loss        | -0.0122  |
| total_timesteps    | 4580825  |
| value_loss         | 0.00748  |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2709     |
| nupdates           | 1900     |
| policy_entropy     | 1.02     |
| policy_loss        | -0.0168  |
| total_timesteps    | 4605075  |
| value_loss         | 0.0085   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2706     |
| nupdates           | 1910     |
| policy_entropy     | 0.954    |
| policy_loss        | 0.000918 |
| total_timesteps    | 4629325  |
| value_loss         | 0.00885  |
---------------------------------


In [ ]:
cc2_toe=Lifecycle(env='unemployment-v1',minimal=False,include_putki=False,mortality=mortality,perustulo=False,randomness=randomness)
cc2_toe.render_distrib(load='results/toe12/distrib_toe12_v',n=1,startn=0,stats_results='results/toe12/distrib_toe12_stats')
cc2_toe.compare_distrib('results/distrib_stats','results/toe12/distrib_toe12_stats')

# Lyhempi työttömyysturva

In [ ]:
cc2_350=Lifecycle(env='unemployment-v1',minimal=False,mortality=mortality,perustulo=False,ansiopvraha_kesto300=250,ansiopvraha_kesto400=350,randomness=randomness)
lyhytmalli='best/350/malli_350'
cc2_350.run_distrib(n=50,startn=0,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=10,batch2=batch2,
                save=lyhytmalli,plot=True,cont=True,start_from=perusmalli,results='results/350/distrib_350',
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage)

In [ ]:
cc2_350=Lifecycle(env='unemployment-v1',minimal=False,include_putki=False,mortality=mortality,perustulo=False,randomness=randomness)
cc2_350.render_distrib(load='results/350/distrib_350_v',n=1,startn=0,stats_results='results/350/distrib_350_stats')
cc2_350.compare_distrib('results/distrib_stats','results/350/distrib_350_stats')